In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Lightweight Python components

To build a component just define a stand-alone python function and then call `kfp.components.func_to_container_op(func)` to convert it to a component that can be used in a pipeline.

There are several requirements for the function:

- The function should be stand-alone. It should not use any code declared outside of the function definition. Any imports should be added inside the main function. Any helper functions should also be defined inside the main function.
- The function can only import packages that are available in the base image. If you need to import a package that's not available you can try to find a container image that already includes the required packages. (As a workaround you can use the module subprocess to run pip install for the required package.)
- **If the function operates on numbers, the parameters need to have type hints. Supported types are [int, float, bool]. Everything else is passed as string.**
- To build a component with multiple output values, use the typing.NamedTuple type hint syntax: `NamedTuple('MyFunctionOutputs', [('output_name_1', type), ('output_name_2', float)])`

In [42]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp

import kubernetes as k8s

In [1]:
# Required Parameters
PROJECT_ID='<ADD GCP PROJECT HERE>'
GCS_BUCKET='gs://<ADD STORAGE LOCATION HERE>'

In [2]:
# Optional Parameters, but required for running outside kubeflow cluster
HOST = '<ADD HOST NAME TO TALK TO KUBEFLOW PIPELINE HERE>'
CLIENT_ID = '<ADD OAuth CLIENT ID USED BY IAP HERE>'
OTHER_CLIENT_ID = '<ADD OAuth CLIENT ID USED TO OBTAIN AUTH CODES HERE>'
OTHER_CLIENT_SECRET = '<ADD OAuth CLIENT SECRET USED TO OBTAIN AUTH CODES HERE>'

## Create client

**If submit outside the kubeflow cluster, need the following**
- `host`: the host name to use to talk to Kubeflow Pipelines, i.e., "https://`<your-deployment>`.endpoints.`<your-project>`.cloud.goog/pipeline"
- `client_id`: The client ID used by Identity-Aware Proxy
- `other_client_id`: The client ID used to obtain the auth codes and refresh tokens.
- `other_client_secret`: The client secret used to obtain the auth codes and refresh tokens.
- For getting `other_client_id` and `other_client_secret`, you'll need to create OAuth client ID credentials of type `Other` according to the tutorial [here](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

```python
client = kfp.Client(host, client_id, other_client_id, other_client_secret)
```

**If you run and submit within the kubeflow cluster**, the following is enough
```python
client = kfp.Client()
```

In [ ]:
# Create kfp client
in_cluster = True
try:
  k8s.config.load_incluster_config()
except:
  in_cluster = False
  pass

if in_cluster:
    client = kfp.Client()
else:
    client = kfp.Client(host=HOST, 
                        client_id=CLIENT_ID,
                        other_client_id=OTHER_CLIENT_ID, 
                        other_client_secret=OTHER_CLIENT_SECRET)

## Start with a simple function

In [3]:
#Define a Python function
def add(a: float, b: float) -> float:
   '''Calculates sum of two arguments'''
   return a + b

Convert the function to a pipeline operation

In [4]:
add_op = comp.func_to_container_op(add)

In [5]:
type(add_op)

function

## A more complicate function with multiple outputs

In [6]:
# Advanced function
# Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple

def my_divmod(dividend: float, 
              divisor: float, 
              output_dir: str = './'
             ) -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    
    '''Divides two numbers and calculate  the quotient and remainder'''
    
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }
    with open(output_dir + 'mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'quotient',
          'numberValue':  float(quotient),
        },{
          'name': 'remainder',
          'numberValue':  float(remainder),
        }]}

    with file_io.FileIO(output_dir + 'mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

In [7]:
my_divmod(100, 7)

MyDivmodOutput(quotient=14, remainder=2)

In [8]:
divmod_op = comp.func_to_container_op(my_divmod)

## Define the pipeline

In [9]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a='a',
   b='7',
   c='17',
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #Passing a task output reference as operation arguments
    #For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
    divmod_task = divmod_op(add_task.output, b, '/')

    #For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax
    result_task = add_op(divmod_task.outputs['quotient'], c)

## Compile the pipeline

In [10]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [11]:
! ls .

00. Setup.ipynb
01. Local Development Quickstart.ipynb
02. Lightweight Python components.ipynb
03. GCP Components Pipeline.ipynb
04. TFX Pipeline from Pre-build Components.ipynb
05. Spark and XGboost Pipeline.ipynb
calc_pipeline.pipeline.zip
mlpipeline-metrics.json
mlpipeline-ui-metadata.json


## Submit the pipeline

In [29]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

experiment = client.create_experiment('python-functions')

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

# Train a keras model

In [33]:
def minist_train(model_file: str, bucket: str) -> str:

    from datetime import datetime
    import tensorflow as tf
    
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    print(model.summary())    
    
    
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    callbacks = [
      tf.keras.callbacks.TensorBoard(log_dir=bucket + '/logs/' + datetime.now().date().__str__()),
      # Interrupt training if `val_loss` stops improving for over 2 epochs
      tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    ]
    
    model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,
              validation_data=(x_test, y_test))
    
    
    model.save(model_file)
    
    from tensorflow import gfile
    
    gcs_path = bucket + "/" + model_file
    
    if gfile.Exists(gcs_path):
        gfile.Remove(gcs_path)
    
    gfile.Copy(model_file, gcs_path)
    
    return gcs_path

In [34]:
minist_train(model_file='mnist_model.h5', 
             bucket=GCS_BUCKET)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
None
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 7s 123us/sample - loss: 0.2175 - acc: 0.9359 - val_loss: 0.1156 - val_acc: 0.9658
Epoch 2/5
60000/60000 [==============================] - 6s 105us/sample -

'gs://kubeflow-pipeline-fantasy-kubeflow1-bucket/mnist_model.h5'

In [35]:
model_train_op = comp.func_to_container_op(func=minist_train, 
                                           base_image="tensorflow/tensorflow:1.15.0-py3")

## Define and submit the pipeline

In [43]:
@dsl.pipeline(
   name='Minist pipeline',
   description='A toy pipeline that performs minist model training.'
)
def minist_pipeline(
    model_file: str = 'mnist_model.h5', 
    bucket: str = GCS_BUCKET
):
    model_train_op(model_file=model_file, bucket=bucket).apply(gcp.use_gcp_secret('user-gcp-sa'))

In [45]:
pipeline_func = minist_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

compiler.Compiler().compile(pipeline_func, pipeline_filename)
#Submit a pipeline run
arguments = {"model_file":"mnist_model.h5",
             "bucket":GCS_BUCKET}
run_name = pipeline_func.__name__ + ' run'
experiment = client.create_experiment('python-functions-minist')

run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

## Submit pipeline directly from pipeline function
# client.create_run_from_pipeline_func(pipeline_func, experiment_name={}, run_name={} arguments={})